In [12]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings

import os

#### Load Data from books

In [26]:
from langchain_community.vectorstores import FAISS

pdf_folder_path = "/Users/harsh_dadwal/Harsh/coding/Hack4skill/Govt_policy_RAG/data"
documents = []
i=1
for file in os.listdir(pdf_folder_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(pdf_folder_path, file)
        loader = PyPDFLoader(pdf_path)
        documents.extend(loader.load())
        print(i)
        i+=1
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
chunked_documents = text_splitter.split_documents(documents)
vectordb = FAISS.from_documents(chunked_documents,OllamaEmbeddings(model='mxbai-embed-large'))
vectordb.save_local("./vectordb",'vector_store')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60


Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 131 0 (offset 0)
Ignoring wrong pointing object 164 0 (offset 0)
Ignoring wrong pointing object 174 0 (offset 0)
Ignoring wrong pointing object 177 0 (offset 0)


61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88


In [27]:
query = "Best way to grow a business?"
docs = vectordb.similarity_search_with_score(query,top_k=5)
print(type(docs[0][0]))
for doc in docs:
    print(f'''{doc[0].page_content} : Score={doc[1]} \n''')

<class 'langchain_core.documents.base.Document'>
173
SOURCES OF BUSINESS FINANCE
8.1 i ntro duction
This chapter provides an overview of 
the various sources from where funds 
can be procured for starting as also for 
running 	a	business. 	It	also	discusses 	
the advantages and limitations of various sources and points out the factors that determine the choice of a 
suitable 	source	of	business 	finance.
It	is	important 	for	any	person	who	
wants to start a business to know about the different sources from 
where	money	can	be	raised.	It	is	also	
important to know the relative merits and demerits of different sources so that choice of an appropriate source 
can	be	made.
8.2 Meaning , nature and Significance  
of BuSineSS  finance
Business is concerned with the 
production and distribution of goods and services for the satisfaction of needs	of	society. 	For	carrying 	out	
various activities, business requires 
money.	Finance, 	therefore, 	is	called	the 
life	blood	of	any	business. 	The	


In [28]:
from langchain_community.llms.ollama import Ollama

embedding=OllamaEmbeddings(model='mxbai-embed-large')
db=FAISS.load_local(folder_path="./vectordb",index_name="vector_store",embeddings=embedding,allow_dangerous_deserialization=True)
llm = Ollama(
    model="phi3:mini"
) 

In [39]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
            <|system|>
            You are a helpful Business Advisor that assists small scale business owners with their queries.<|end|>
            <|user|>
            Answer the following question as simply as possible based on the given context ONLY if it is RELEVANT to the Question in less than 100 words if it is IRRELEVANT tell me I don't know.
            Do not mention the context in the answer just use it as a knowledge base
            {context}
            Question:
            {input}
            <|end|>
            <|assistant|>

"""
)

In [40]:
from langchain.chains.combine_documents import create_stuff_documents_chain

docs_chain = create_stuff_documents_chain(llm, prompt)
retriever = db.as_retriever()

In [41]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    print((type(store[session_id])))
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [42]:
response = conversational_rag_chain.invoke({"input": "Best way to grow a business?"},{"configurable":{"session_id": "1"}})
print(response['answer'])

<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>
 As an artificial intelligence, I don't have personal experiences or opinions. However, based on the information provided and best practices in finance management theory as of my last update, here is what experts typically advise for growing a business effectively through capital structure decisions:

1. **Understand Your Capital Structure** - Evaluate your current debt level compared to equity within the firm's balance sheet. This gives you an idea if taking on more debt could be risky or beneficial at this time. 

2. **Assess Cash Flow and Profitability Needs** – Determine whether additional capital is required for growth initiatives like expanding into new markets, upgrading technology, hiring staff, etc., which will help you decide if debt financing makes sense or if seeking equity investment might be a better route.

3. **Maintain Balance Between Debt and Equity** - Aim for an optimal capital structure where the cost

In [43]:
response = conversational_rag_chain.invoke({"input": "what are some government schemes that can help me"},{"configurable":{"session_id": "1"}})
print(response['answer'])

<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>
 Several governments around the world offer programs and financial assistance aimed at helping MSMEs grow by overcoming their financing hurdles among other challenges they face, like difficulties in procuring raw materials or protecting themselves from exploitative lending practices. While I don't have real-time updates on schemes outside my last knowledge update (March 2021), here are several types of government initiatives commonly available:

**Financial Subsidies and Grants** - Some governments provide grants that can be used to finance business operations, especially for startups or small enterprises in specific sectors. Examples include technology development funds or rural innovation subsidies. The availability of such schemes often depends on the industry you're operating within and your country’s current economic focuses (e.g., green energy).

**Tax Incentives and Credits** - Tax relief measures, including reduce

In [44]:
print(response['context'])

[Document(metadata={'source': '/Users/harsh_dadwal/Harsh/coding/Hack4skill/Govt_policy_RAG/data/kebs109.pdf', 'page': 15}, page_content='213\nMSME AND BUSINESS ENTREPRENEURSHIP\nRole of small business in rural India: Small business units provide multiple \nsource of income, in wide range of non-agricultural activities and provide employment opportunities in rural areas, especially for the traditional artisan and weaker sections of the society.\nEntrepreneur:  The terms ‘entrepreneur,’ ‘entrepreneurship’ and ‘enterprises’ \ncan be understood by drawing an analogy with the structure of a sentence in \nEnglish language. Entrepreneur is the person (the subject), entrepreneurship is the process (the verb) and enterprise is the creation of the person and the output of the process (the object).\nEXERCISES\nVery Short Answer Questions\n1. Which year the MSMED Act passed?\n2. What is the micro enterprise.\n3. What is a cottage industry?\n4. What is meant by Village and Khadi Industry?\n5. Give 